In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from sentida import Sentid
from afinn import Afinn

# Read the data
df = pd.read_csv('../data/letters.csv')

In [ ]:

afinn = Afinn(language='da')
scores = [afinn.score(text) for text in df.text]
df['sentiment_score'] = scores

df_sentences = pd.read_csv('../data/sentences.csv')
scores = [afinn.score(str(sentence)) for sentence in df_sentences.sentence]
df_sentences['afinn_score'] = scores

In [ ]:
# full text sentida
sentida_instance = Sentida()
sentida_scores = [sentida_instance.sentida(str(text),output="total") for text in df.text]
df['sentida_score'] = sentida_scores

In [ ]:
sentida_instance = Sentida()
# 12 seconds
sentida_scores = [sentida_instance.sentida(str(sentence),output="total") for sentence in df_sentences.sentence]
df_sentences['sentida_score'] = sentida_scores

In [ ]:
# saving sentiment scored sentences
df_sentences.to_csv('../data/sentiment_scored_sentences.csv', index=False)

In [ ]:
# avg afinn_score per letter in df_sentences
df_agg_sentiment = df_sentences.groupby('letter_id').afinn_score.mean()
df['affin_score_sentenceavg'] = df_agg_sentiment

df_agg_sentida = df_sentences.groupby('letter_id').sentida_score.mean()
df['sentida_score_sentenceavg'] = df_agg_sentida

In [ ]:
# plot sentiment score over time for date
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])

fig, ax = plt.subplots(1,4,figsize=(15,7))
ax[0].plot(df.date, df.sentiment_score, color='blue', alpha=0.5, linewidth=0.5)
# title on ax 1
ax[0].set_title('Afinn score full text')
ax[1].plot(df.date, df.affin_score_sentenceavg, color='red', alpha=0.5, linewidth=0.5)
ax[1].set_title('Afinn score sentence avg')
ax[2].plot(df.date, df.sentida_score_sentenceavg, color='green', alpha=0.5, linewidth=0.5)
ax[2].set_title('Sentida score sentence avg')
ax[3].plot(df.date, df.sentida_score, color='purple', alpha=0.5, linewidth=0.5)
ax[3].set_title('Sentida score ')

In [ ]:
from danlp.models import load_bert_emotion_model
classifier = load_bert_emotion_model()

In [ ]:
# add bert emotion score per letter
bert_scores = [classifier.predict(str(text)) for text in df.text]
df['bert_emotion_score'] = bert_scores

In [ ]:
df.to_csv('../data/sentiment_scored_letters.csv', index=False)